# Multi-Layer Perceptron (MLP) - Large & Complex Data Set


In this notebook we will train MLP classifier on a **large & non-linear data set**. We will use an image classification (multi-class) problem for experimentation. 

For an analytical understanding, we will compare the performance of the MLP with the SVM and Logistic Regression classifiers.


Due to the non-linearity of the features (i.e., pixels), we will use the Gaussian Radial Basis Function (RBF) Kernel based Support Vector Machine (SVM). Previously we have seen that the Gaussian RBF Kernel based SVM performs better than Softmax regression classifier.

To expedite the training time, we use dimensionality reduction technique (Principle Component Analysis) to project the features into a smaller dimension.

Our goal is to investigate whether MLP outperforms the Gaussian RBF Kernel SVM on a very large complex data set.

We conduct the following experiments.


## Experiments

- Experiment 1: Multi-Layer Perceptron 
- Experiment 2: Multi-Layer Perceptron + PCA
- Experiment 2: Support Vector Machine (SVC with RBF Kernel) + PCA
- Experiment 3: Logistic Regression (Softmax Regression) + PCA


## Dataset: MNIST


We will use the MNIST dataset, which is a set of 70,000 small images of digits handwritten by high school students and employees of the US Census Bureau. Each image is labeled with the digit it represents.


There are 70,000 images. Each image is 28x28 pixels, and each feature simply represents one pixel’s intensity, from 0 (white) to 255 (black).

Thus, each image has 784 features. 

In [1]:
import warnings
import time
import numpy as np
import pandas as pd
from scipy.io import loadmat



from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Load Data and Create Data Matrix (X) and the Label Vector (y)

First load the data and explore the feature names, target names, etc.

We may load the data from a local folder or load it directly from cloud using Scikit-Learn.

In [2]:
# Load the data from the local folder "data"
mnist = loadmat('data/mnist-original.mat')

#Create the data Matrix X and the target vector y
X = mnist["data"].T.astype('float64')
y = mnist["label"][0].astype('int64')


# Load data using Scikit-Learn
# mnist = fetch_openml('mnist_784', cache=False)

# X = mnist["data"].astype('float64')
# y = mnist["target"].astype('int64')


print("\nNo. of Samples: ", X.shape)
print("No. of Labels: ", y.shape)

print("\nX type: ", X.dtype)
print("y type: ", y.dtype)


No. of Samples:  (70000, 784)
No. of Labels:  (70000,)

X type:  float64
y type:  int64


## Split Data Into Training and Test Sets

We use sklearn's train_test_split function to spilt the dataset into training and test subsets. The data is shuffled by default before splitting.

This function splits arrays or matrices into random train and test subsets.

For the reproducibility of the results, we need to use the same seed for the random number generator. The seed is set by the "random_state" parameter of the split function.

However, in repeated experiments if we don't want to use the same train and test subsets, then we drop the "random_state" parameter from the funtion.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Optimization Using Dimensionaly Reduction

We can optimize the running-time of the MLP model by reducing the number of features. Our assumption is that the essence or core content of the data does not span along all dimensions. The technique for reducing the dimension of data is known as dimensionality reduction.

For a gentle introduction to various dimensionality reduction technique, see the notebook "Dimensionality Reduction" in the Github repository.

We will use the **Principle Component Analysis (PCA)** dimensionality reduction technique to project the MNIST dataset (784 features) to a lower dimensional space by retaining maximum variance. 

The goal is to see the improvement in training time due to this dimensionality reduction.

Before we apply the PCA, we need to standardize the data.

## Standardize the Data

PCA is influenced by scale of the data. Thus we need to scale the features of the data before applying PCA. 

For understanding the negative effect of not scaling the data, see the following post:

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#sphx-glr-auto-examples-preprocessing-plot-scaling-importance-py

Note that we fit the scaler on the training set and transform on the training and test set. 

In [4]:
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train)

# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Apply PCA

While applying PCA we can set the number of principle components by the "n_components" attribute. But more importantly, we can use this attribute to determine the % of variance we want to retain in the extracted features.

For example, if we set it to 0.95, sklearn will choose the **minimum number of principal components** such that 95% of the variance is retained.

In [5]:
%%time
pca = PCA(n_components=0.95)

pca.fit(X_train)

CPU times: user 13.1 s, sys: 920 ms, total: 14 s
Wall time: 4.5 s


PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

## Number of Principle Components

We can find how many components PCA chose after fitting the model by using the following attribute: n_components_

We will see that 95% of the variance amounts to **315 principal components**.

In [6]:
print("Number of Principle Components: ", pca.n_components_)  

Number of Principle Components:  330


## Apply the Mapping (Transform) to both the Training Set and the Test Set

In [7]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

## Experiments

We will conduct the following experiments.

- Experiment 1: Multi-Layer Perceptron 
- Experiment 2: Multi-Layer Perceptron + PCA
- Experiment 2: Support Vector Machine (SVC with RBF Kernel) + PCA
- Experiment 3: Logistic Regression (Softmax Regression) + PCA


## Experiment 1: MLP 

First we train the MLP without applying PCA on the data. So we use 784 features.

See the notebook "Perceptron-MLP-Nonlinear Data" for a discussion on various solvers that are used by MLP and the hyperparameters.


## Model Selection: Hyperparameter Tunining


We use Scikit-Learn’s GridSearchCV to search the combinations of hyperparameter values that provide the best performance.

We did not tune all hyperparameters. Also note that the range of values for the hyperparameters should be chosen thoughtfully. We used prior knowledge based on theoretical and empirical understanding to choose the range of values.


We tune the following hyperparameters of the GridSearchCV model.


- hidden_layer_sizes: only one hidden layer is enough to produce optimal accuracy. Number of neurons should be raged from 100 to 200. 

- activation : ‘logistic’ and ‘relu’. 

- solver : Since the dataset is large, we did not use 'lbfgs' as it is suitable for small dataset for faster convergence. The two solvers used for tuning are: ‘sgd’ and ‘adam’. 

- alpha : L2 penalty (regularization term) parameter.

- learning_rate : ‘constant’ and ‘adaptive’. Note that the solver 'adam' works with 'constant' learning rate as it implements a learning schedule to decrease learning rate dynamically.

- learning_rate_init : The initial learning rate used. It controls the step-size in updating the weights for 'adam' and 'sgd'. The solver 'adam' requires smaller initial learning rate (0.001), while 'sgd' requires a slightly larger value (0.1).


For performing the hyperparameter tunining, we set the following attributes of the model to fixed values.

- The "early_stopping" parameter is set to True to prevent overfitting (that is caused by overtraining).

- If we use early stopping, then we should also set the "n_iter_no_change" to a suitable value. It defines the maximum number of epochs to not meet tol improvement. The default is 10.

- tol : Tolerance for the optimization.


### Hyperparameters for Faster Optimizers


#### Optimizer: SGD

- momentum : float, default 0.9

        Momentum for gradient descent update. Should be between 0 and 1. Only used when solver=’sgd’.

- nesterovs_momentum : boolean, default True

        Whether to use Nesterov’s momentum. Only used when solver=’sgd’ and momentum > 0.
        
  
#### Optimizer: Adam
- beta_1 : float, optional, default 0.9

        Exponential decay rate for estimates of first moment vector in adam, should be in [0, 1). Only used when solver=’adam’

- beta_2 : float, optional, default 0.999

        Exponential decay rate for estimates of second moment vector in adam, should be in [0, 1). Only used when solver=’adam’

- epsilon : float, optional, default 1e-8

        Value for numerical stability in adam. Only used when solver=’adam’


More info: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

For a brief discussion on the faster optimizers (momentum, adam), see the 2nd notebook on MLP.

In [ ]:
%%time

param_grid = {'hidden_layer_sizes': [(100,), (150,), (200,)], "solver":['sgd', 'adam'], 
              'learning_rate_init': (0.1, 0.01, 0.001), "alpha": (0.1, 0.01),
              'activation': ['logistic', 'relu']}

clf_mlp = MLPClassifier(early_stopping=True, n_iter_no_change=10, tol=1e-5, max_iter=500, random_state=1)

clf_mlp_cv = GridSearchCV(clf_mlp, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
clf_mlp_cv.fit(X_train, y_train)


params_optimal_mlp = clf_mlp_cv.best_params_

print("Best Score (accuracy): %f" % clf_mlp_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal_mlp)
print("\n")


Fitting 5 folds for each of 72 candidates, totalling 360 fits
- [Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
- [Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 23.8min
- [Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 177.4min
- [Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 298.9min finished

Best Score (accuracy): 0.973393

Optimal Hyperparameter Values:  {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (200,), 'learning_rate_init': 0.001, 'solver': 'adam'}


Wall time: 5h 31s

In [8]:
%%time

t0 = time.time()
mlp_clf = MLPClassifier(hidden_layer_sizes=(200,), max_iter=200, alpha=0.01,
                    solver='adam', verbose=True, tol=1e-5, random_state=1, 
                    learning_rate='constant', learning_rate_init=0.001, activation='relu',
                    early_stopping=True, n_iter_no_change=10)

mlp_clf.fit(X_train, y_train)

t1 = time.time()

duration_mlp = t1 - t0
print("The MLP takes {:.1f}s.".format(duration_mlp))

print("No. of Iterations:", mlp_clf.n_iter_ )

y_train_predicted = mlp_clf.predict(X_train)

train_accuracy_mlp = np.mean(y_train_predicted == y_train)
print("\nTraining Accuracy: ", train_accuracy_mlp)

Iteration 1, loss = 0.32298691
Validation score: 0.949464
Iteration 2, loss = 0.13119739
Validation score: 0.962321
Iteration 3, loss = 0.09264147
Validation score: 0.965714
Iteration 4, loss = 0.06617721
Validation score: 0.970357
Iteration 5, loss = 0.05117022
Validation score: 0.967321
Iteration 6, loss = 0.04344897
Validation score: 0.971607
Iteration 7, loss = 0.03361663
Validation score: 0.972500
Iteration 8, loss = 0.02845721
Validation score: 0.972321
Iteration 9, loss = 0.02450681
Validation score: 0.972857
Iteration 10, loss = 0.02429279
Validation score: 0.974464
Iteration 11, loss = 0.02010919
Validation score: 0.974643
Iteration 12, loss = 0.01814253
Validation score: 0.976607
Iteration 13, loss = 0.01674399
Validation score: 0.974464
Iteration 14, loss = 0.01568636
Validation score: 0.976429
Iteration 15, loss = 0.01513316
Validation score: 0.975179
Iteration 16, loss = 0.01716468
Validation score: 0.964643
Iteration 17, loss = 0.06938609
Validation score: 0.963036
Iterat

## Experiment 1: Evaluate MLP  on Test Data

In [9]:
%%time
y_test_predicted = mlp_clf.predict(X_test)

accuracy_score_test_mlp = np.mean(y_test_predicted == y_test)
print("\nTest Accuracy: ", accuracy_score_test_mlp)

print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))

print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))


Test Accuracy:  0.9766428571428571

Test Confusion Matrix:
[[1296    0    1    0    1    1    9    2    2    0]
 [   0 1587    8    2    1    0    1    3    2    0]
 [   1    5 1318    8    5    0    2    2    5    2]
 [   0    3   11 1387    0   11    0    6    6    3]
 [   1    2    6    0 1321    1    6    5    2   18]
 [   1    0    3   10    0 1241    7    2   12    4]
 [   3    2    2    0    2    4 1383    0    1    0]
 [   2    3    7    2    3    1    0 1428    2   13]
 [   0    3    4    9    2    3    4    6 1355    4]
 [   4    3    3    7   17    3    1   16    8 1357]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1312
           1       0.99      0.99      0.99      1604
           2       0.97      0.98      0.97      1348
           3       0.97      0.97      0.97      1427
           4       0.98      0.97      0.97      1362
           5       0.98      0.97      0.98      1280
      

## Expriment 2: MLP + PCA

In [10]:
%%time

t0 = time.time()
mlp_clf_pca = MLPClassifier(hidden_layer_sizes=(200,), max_iter=200, alpha=0.01,
                    solver='adam', verbose=True, tol=1e-5, random_state=1, 
                    learning_rate='constant', learning_rate_init=0.001, activation='relu',
                    early_stopping=True, n_iter_no_change=10)


mlp_clf_pca.fit(X_train_pca, y_train)

t1 = time.time()

duration_mlp_pca = t1 - t0
print("The PCA+MLP takes {:.1f}s.".format(duration_mlp_pca))

print("No. of Iterations:", mlp_clf_pca.n_iter_ )

y_train_predicted = mlp_clf_pca.predict(X_train_pca)

train_accuracy_mlp = np.mean(y_train_predicted == y_train)
print("\nTraining Accuracy: ", train_accuracy_mlp)

Iteration 1, loss = 0.49712015
Validation score: 0.935357
Iteration 2, loss = 0.19141485
Validation score: 0.948750
Iteration 3, loss = 0.14316349
Validation score: 0.957679
Iteration 4, loss = 0.11065116
Validation score: 0.964286
Iteration 5, loss = 0.08852195
Validation score: 0.967679
Iteration 6, loss = 0.07122619
Validation score: 0.967857
Iteration 7, loss = 0.06032630
Validation score: 0.969821
Iteration 8, loss = 0.05056308
Validation score: 0.973214
Iteration 9, loss = 0.04944693
Validation score: 0.970893
Iteration 10, loss = 0.04480907
Validation score: 0.972500
Iteration 11, loss = 0.03962612
Validation score: 0.974286
Iteration 12, loss = 0.04125004
Validation score: 0.973750
Iteration 13, loss = 0.03956243
Validation score: 0.972500
Iteration 14, loss = 0.03339130
Validation score: 0.974286
Iteration 15, loss = 0.02977469
Validation score: 0.974107
Iteration 16, loss = 0.02981106
Validation score: 0.975000
Iteration 17, loss = 0.04301179
Validation score: 0.973214
Iterat

## Experiment 2: Evaluate MLP + PCA on Test Data

In [11]:
%%time
y_test_predicted = mlp_clf_pca.predict(X_test_pca)

accuracy_score_test_mlp_pca = np.mean(y_test_predicted == y_test)
print("\nTest Accuracy: ", accuracy_score_test_mlp_pca)

print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))

print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))


Test Accuracy:  0.9735714285714285

Test Confusion Matrix:
[[1297    0    2    0    0    1    5    4    3    0]
 [   0 1584   11    1    2    0    0    2    3    1]
 [   3    5 1312    5    3    1    5    8    4    2]
 [   1    3   11 1383    0   16    0    4    5    4]
 [   2    1    7    0 1320    2    6    3    2   19]
 [   1    1    3   12    1 1233    9    4   12    4]
 [   4    2    3    0    4    6 1375    2    1    0]
 [   1    2    9    3    5    2    1 1421    4   13]
 [   1    6    6    8    3    7    4    4 1345    6]
 [   3    2    3    8   17    5    0   13    8 1360]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1312
           1       0.99      0.99      0.99      1604
           2       0.96      0.97      0.97      1348
           3       0.97      0.97      0.97      1427
           4       0.97      0.97      0.97      1362
           5       0.97      0.96      0.97      1280
      

## Experiment 2: SVC (RBF Kernel) + PCA

In [12]:
%%time

t0 = time.time()
svm_clf_pca = SVC(C=1, gamma=0.001)
svm_clf_pca.fit(X_train_pca, y_train)

t1 = time.time()

duration_svm_pca = t1 - t0
print("The PCA+SVM takes {:.1f}s.".format(duration_svm_pca))

The PCA+SVM takes 193.4s.
CPU times: user 3min 12s, sys: 591 ms, total: 3min 13s
Wall time: 3min 13s


## Experiment 2: Evaluate SVC (RBF Kernel) + PCA on Test Data

In [13]:
%%time

y_test_predicted = svm_clf_pca.predict(X_test_pca)

accuracy_score_test_svm = np.mean(y_test_predicted == y_test)
print("\nTest Accuracy: ", accuracy_score_test_svm)

print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))

print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))


Test Accuracy:  0.9652857142857143

Test Confusion Matrix:
[[1297    0    1    0    0    1    8    2    3    0]
 [   0 1574   11    2    1    0    0    8    4    4]
 [   3    4 1301   11    6    1    7    8    5    2]
 [   0    4   15 1366    2    9    0   15   11    5]
 [   3    2    8    1 1312    1    5    7    3   20]
 [   2    1    4   20    3 1217   13    6   10    4]
 [   5    1    3    0    2    7 1369    7    3    0]
 [   5    3   13    3    7    1    0 1411    1   17]
 [   0   11    8   13    4    8    6    4 1334    2]
 [   3    5    6   16   23    5    1   24    3 1333]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1312
           1       0.98      0.98      0.98      1604
           2       0.95      0.97      0.96      1348
           3       0.95      0.96      0.96      1427
           4       0.96      0.96      0.96      1362
           5       0.97      0.95      0.96      1280
      

## Experiment 3: Logistic Regression (Softmax Regression) + PCA

We use the best performing solver (i.e., lbfgs) from previous notebook to train the logistic regression model on the PCA transformed data.

In [14]:
%%time

t0 = time.time()

softmax_reg_pca = LogisticRegression(solver='lbfgs', multi_class='multinomial')

softmax_reg_pca.fit(X_train_pca, y_train)

t1 = time.time()

duration_lr_pca = t1 - t0
print("The PCA + Logistic Regression takes {:.1f}s.".format(duration_lr_pca))

The PCA + Logistic Regression takes 4.0s.
CPU times: user 15.3 s, sys: 344 ms, total: 15.6 s
Wall time: 4.01 s


/Users/hasan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


## Experiment 4: Evaluate Softmax Regression + PCA on Test Data

In [15]:
print("No. of Iterations:", softmax_reg_pca.n_iter_ )


y_test_predicted = softmax_reg_pca.predict(X_test_pca)


accuracy_score_test_lr = np.mean(y_test_predicted == y_test)
print("\nTest Accuracy: ", accuracy_score_test_lr)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))

No. of Iterations: [100]

Test Accuracy:  0.9220714285714285

Test Confusion Matrix:
[[1270    0    4    3    5    9   11    3    7    0]
 [   0 1557   10    7    2    4    0    2   16    6]
 [  10   22 1211   24   15    2   22   13   24    5]
 [   3   14   27 1283    1   44    3    8   33   11]
 [   4    8    9    2 1267    3   10    4    9   46]
 [   9    9   13   43   12 1111   21    8   40   14]
 [   9    4    8    1    9   15 1342    2    5    2]
 [   5    3   23    4   14    1    1 1363    1   46]
 [   6   28   12   38    7   32   10    3 1239   15]
 [   8    5    4   25   46   11    1   43   10 1266]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      1312
           1       0.94      0.97      0.96      1604
           2       0.92      0.90      0.91      1348
           3       0.90      0.90      0.90      1427
           4       0.92      0.93      0.92      1362
           5       0.90      0.87  

# Summary of Results from 3 Experiments

In [17]:
data = [["MLP", accuracy_score_test_mlp, duration_mlp], 
        ["MLP + PCA", accuracy_score_test_mlp_pca, duration_mlp_pca], 
        ["SVM(RBF) + PCA", accuracy_score_test_svm, duration_svm_pca],
        ["Softmax + PCA", accuracy_score_test_lr, duration_lr_pca]]

pd.DataFrame(data, columns=["Classifier", "Test Accuracy", "Running-Time"])


,Classifier,Test Accuracy,Running-Time
0,MLP,0.976643,47.365682
1,MLP + PCA,0.973571,25.742746
2,SVM(RBF) + PCA,0.965286,193.355095
3,Softmax + PCA,0.922071,4.012608


## Comparative Understanding

We have done 4 experiments using MLP (without PCA), MLP (with PCA), Kernel SVM (with PCA) and Logistic Regression (with PCA) classifiers.

We make following observations.
- The MLP outperforms other two classifiers.
- MLP with PCA performs slightly less than MLP without PCA, but it is faster.
- Understandably logistic regression performed poorly due to the non-linear nature of the data. However, it is faster.
- MLP (with PCA) is faster than the Kernel SVM. Because the dual SVM optimization complexity is $O(N^2d)$.

### Thus, for large non-linear data set (e.g., image classification) MLP performs better than the RBF kernel based SVM.